# Export list of references

Author: [José R. Ferrer-Paris](https://github.com/jrfep)

Date: July 2024

This Jupyter Notebook includes Python code to download data from the Fireveg Database to a local folder. The output is a workbook in XLSX format with the list of references used in the `litrev` schema of the database.

**Please note:**
<div class="alert alert-warning">
    This code is intended for internal project management and  is documented for the sake of reproducibility.<br/>
    🛂 Only users contributing directly to the project have access to the credentials for data download/upload. 
</div>

## Set-up

### Load modules
We are using Python for this. Start your session and load the packages.


In [1]:
# work with paths in operating system
from pathlib import Path
import os
import sys

# datetime support
import datetime
# For database connection
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor

# Pandas for calculations
import pandas as pd
# Regular expressions
import re
# Pyprojroot for easier handling of working directory
import pyprojroot

### Define paths for input and output

Define project directory using the `pyprojroot` functions, and add this to the execution path.

In [2]:
repodir = pyprojroot.find_root(pyprojroot.has_dir(".git"))
sys.path.append(str(repodir))

Declare and create an output data folder

In [3]:
outputdir = repodir / "data" / "output-report"
if (not os.path.exists(outputdir)):
    os.mkdir(outputdir)
os.listdir(outputdir)

['fireveg-trait-records-model.xlsx',
 'fireveg-trait-records-curation.xlsx',
 'fireveg-trait-report-model.xlsx',
 'fireveg-db-references.xlsx',
 'fireveg-field-report-model.xlsx',
 'fireveg-field-records.csv',
 'fireveg-trait-records.csv']

### Import own functions
Load functions from `lib` folder, we will use a function to read db credentials, one for executing database queries and three functions for extracting data from the reference description string

In [4]:
from lib.parseparams import read_dbparams
from lib.firevegdb import dbquery
from lib.firevegrefs import extract_year,extract_authors,extract_rest

### Database credentials

🤫 We use a folder named "secrets" to keep the credentials for connection to different services (database credentials, API keys, etc). This checked this folder in our `.gitignore` so that its content are not tracked by git and not exposed. Future users need to copy the contents of this folder manually.

We read database credentials stored in a `database.ini` file using our own `read_dbparams` function.

In [5]:
dbparams = read_dbparams(repodir / 'secrets' / 'database.ini', section='fireveg-db-v1.1')

## Database query

Database connection and query using functions defined in the project library.

In [6]:
ref_info = dbquery("SELECT * FROM litrev.ref_list ", dbparams)

## Create data frame and add columns
We transform the query result to a data frame and add three columns with data extracted from the ref_cite string.

In [7]:
df=pd.DataFrame(ref_info,columns=ref_info[1].keys())

In [8]:
df['date']=df.apply(lambda row : extract_year(row['ref_cite']), axis = 1)
df['authors']=df.apply(lambda row : extract_authors(row['ref_cite']), axis = 1)
df['ref_info']=df.apply(lambda row : extract_rest(row['ref_cite']), axis = 1)

In [9]:
df

,ref_code,ref_cite,alt_code,date,authors,ref_info
0,austraits-6.0.0,"Falster, D., Gallagher, R., Wenk, E., & Sauque...",austraits-6.0.0,2024,"Falster, D., Gallagher, R., Wenk, E., & Sauque...",. AusTraits: a curated plant trait database fo...
1,Auld Keith Bradstock 2000,"Auld, Tony D.; Keith, David A.; Bradstock, Ros...",Auld_2000,2000,"Auld, Tony D.; Keith, David A.; Bradstock, Ros...",{Patterns in longevity of soil seedbanks in f...
2,Baker 2019,"Baker, Andrew (2019) {Unpublished data: Post-f...",Baker_2019,2019,"Baker, Andrew",{Unpublished data: Post-fire fire-response su...
3,Burrows Wardell-Johnson Ward 2008,"Burrows, Neil D; Wardell-Johnson, Grant; Ward,...",Burrows_2008,2008,"Burrows, Neil D; Wardell-Johnson, Grant; Ward,...",{Post-fire juvenile period of plants in south...
4,Burrows 2020,"Burrows, Neil (2020) {Data from 'Plant Ecologi...",Burrows_2020,2020,"Burrows, Neil",{Data from 'Plant Ecological Attributes' them...
...,...,...,...,...,...,...
342,RP RFA Victoria - East Gippsland,RP RFA Victoria - East Gippsland. Environment ...,NSWFFRD-RP-ref-R41,1999,RP RFA Victoria - East Gippsland. Environment ...,RP RFA Victoria - East Gippsland. Environment ...
343,RP RFA Tasmania,RP RFA Tasmania. Tasmanian-Commonwealth RFA Ba...,NSWFFRD-RP-ref-R42,,RP RFA Tasmania. Tasmanian-Commonwealth RFA Ba...,". Tasmanian Public Land Use Commission, Nov 1996."
344,RP Threatened Eucalypts,"RP Threatened Eucalypts. Draft Recovery Plan, ...",NSWFFRD-RP-ref-R43,200,"RP Threatened Eucalypts. Draft Recovery Plan, ...","RP Threatened Eucalypts. Draft Recovery Plan, ..."
345,RP Boronia granitica,RP Boronia granitica. RP Draft for public comm...,NSWFFRD-RP-ref-R44,200,RP Boronia granitica. RP Draft for public comm...,RP Boronia granitica. RP Draft for public comm...


## Export to excel workbook
Here we use a simple excel export function to save the results:

In [10]:
df.to_excel(outputdir / "fireveg-db-references.xlsx") 

## Where is the output?
<div class="alert alert-info">
We uploaded this and other workbooks to report on the database content in a open access repository at:

> Ferrer-Paris, José R.; Keith, D A (2024). Fire Ecology Traits for Plants: Database exports. figshare. Dataset. https://doi.org/10.6084/m9.figshare.24125088
</div>

## That is it for now!

✅ Job done! 😎👌🔥

You can go to the next steps: 
- [Create XLSX output of litrev records](./Create-xlsx-output-litrev-records.ipynb).
- [Create XLSX output of litrev records for curation](./Create-xlsx-output-curation-litrev-records.ipynb).
- [Create XLSX output of a summary of litrev records](./Create-xlsx-output-summary-litrev.ipynb).

Or you can:
- go [back home](../README.md),
- continue navigating the repo on [GitHub](https://github.com/ces-unsw-edu-au/fireveg-db-exports)
- continue exploring the repo on [OSF](https://osf.io/h96q2/).
- visit the database at <http://fireecologyplants.net>